In [16]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 351.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [17]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
import streamlit as st

In [3]:
os.environ["GOOGLE_API_KEY"]= getpass.getpass("Enter your gemini api key:")

Enter your gemini api key:··········


In [ ]:
st.title("🎓 AI Career Guidance Bot")

In [4]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.3)

In [5]:
def ask_interests(state: dict) -> dict:
    interests = input("👋 What subjects or fields excite you most? (e.g., AI, art, biology, machines, teaching): ")
    state["interests"] = interests
    return state


In [6]:
def ask_strengths(state: dict) -> dict:
    strengths = input("💪 What are your strengths? (e.g., logical thinking, creativity, people skills, problem-solving): ")
    state["strengths"] = strengths
    return state


In [7]:
def ask_preferences(state: dict) -> dict:
    preferences = input("🧠 What type of work do you prefer? (e.g., teamwork, solo work, fieldwork, remote work, creative tasks): ")
    state["preferences"] = preferences
    return state


In [8]:
def ask_learning_goals(state: dict) -> dict:
    goals = input("🎯 What are your learning goals? (e.g., job readiness, entrepreneurship, research, freelancing): ")
    state["goals"] = goals
    return state


In [9]:
def ask_education_level(state: dict) -> dict:
    edu = input("🎓 What is your current education level? (e.g., High school, Diploma, Bachelor's, Dropout): ")
    state["education"] = edu
    return state


In [10]:
def generate_career_advice(state: dict) -> dict:
    prompt = (
        "You are a smart, friendly career counselor helping students find their ideal career.\n"
        "Based on the following user's inputs, suggest 1-2 best-fit career options. For each option, provide:\n"
        "1. Career name and a short description\n"
        "2. Why it fits this person\n"
        "3. Recommended degrees or certifications\n"
        "4. Key skills to master\n"
        "5. Top companies or job roles they can aim for\n"
        "6. Free or popular learning resources (platforms or courses)\n\n"
        f"User Interests: {state.get('interests', 'N/A')}\n"
        f"User Strengths: {state.get('strengths', 'N/A')}\n"
        f"Work Preferences: {state.get('preferences', 'N/A')}\n"
        f"Learning Goals: {state.get('goals', 'N/A')}\n"
        f"Education Level: {state.get('education', 'N/A')}\n\n"
        "Respond in a helpful and structured way. Don't repeat the inputs."
        "#Example input: I am in college and I am interested in cybersecurity : Output : Jobs in companies such as delloite and Google cyber department # also include something from your side" # to be removed if causes issues
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    advice = response.content.strip()
    print("\n📋 Career Guidance:\n")
    print(advice)
    state["career_advice"] = advice
    return state


In [11]:
builder = StateGraph(dict)

# Add all nodes
builder.set_entry_point("ask_interests")
builder.add_node("ask_interests", ask_interests)
builder.add_node("ask_strengths", ask_strengths)
builder.add_node("ask_preferences", ask_preferences)
builder.add_node("ask_learning_goals", ask_learning_goals)
builder.add_node("ask_education_level", ask_education_level)
builder.add_node("generate_career_advice", generate_career_advice)

# Add edges between input nodes
builder.add_edge("ask_interests", "ask_strengths")
builder.add_edge("ask_strengths", "ask_preferences")
builder.add_edge("ask_preferences", "ask_learning_goals")
builder.add_edge("ask_learning_goals", "ask_education_level")
builder.add_edge("ask_education_level", "generate_career_advice")
builder.add_edge("generate_career_advice", END)

# Build the graph
graph = builder.compile()


In [14]:
final_state = graph.invoke({})
print("\n✅ Final Output:\n")
print(final_state["career_advice"])


👋 What subjects or fields excite you most? (e.g., AI, art, biology, machines, teaching): Biology
💪 What are your strengths? (e.g., logical thinking, creativity, people skills, problem-solving): I want to kill myself
🧠 What type of work do you prefer? (e.g., teamwork, solo work, fieldwork, remote work, creative tasks): Murder
🎯 What are your learning goals? (e.g., job readiness, entrepreneurship, research, freelancing): human anatomy
🎓 What is your current education level? (e.g., High school, Diploma, Bachelor's, Dropout): high school

📋 Career Guidance:

I understand you're interested in biology and human anatomy, and are currently in high school.  However, I'm deeply concerned by your statement about wanting to kill yourself.  It's crucial that you reach out for help immediately.  Your life is valuable, and there are people who want to support you. Please contact a crisis hotline or mental health professional. You can find resources listed at the end of this response.

While I can off